In [2]:
from __future__ import division,print_function,absolute_import
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [9]:
learning_rate=0.001
num_steps=500
batch_size=128
display_step=10

num_input=784
num_classes=10
dropout=0.75

X=tf.placeholder(tf.float32,[None,num_input])
Y=tf.placeholder(tf.float32,[None,num_classes])
keep_prob=tf.placeholder(tf.float32)

In [10]:
def conv2d(x,W,b,strides=1):
    #Conv2D wrapper,with bias and relu activation
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x=tf.nn.bias_add(x,b)
    return tf.nn.relu(x)
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],
                         padding='SAME')
#Create model
def conv_net(x,weights,biases,dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x=tf.reshape(x,shape=[-1,28,28,1])
    conv1=conv2d(x,weights['wc1'],biases['bc1'])
    conv1=maxpool2d(conv1,k=2)
    
    conv2=conv2d(conv1,weights['wc2'],biases['bc2'])
    conv2=maxpool2d(conv2,k=2)
    #Fully connected layer
    fc1=tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1=tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1=tf.nn.relu(fc1)
    #Apply Dropout
    fc1=tf.nn.dropout(fc1,dropout)
    #Output,class prediction
    out=tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out


In [13]:
weights={
    #5x5 conv,1 input,32 output
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    'out':tf.Variable(tf.random_normal([1024,num_classes]))
    
}
biases={
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([num_classes]))
}
#Construct model
logits=conv_net(X,weights,biases,keep_prob)
prediction=tf.nn.softmax(logits)
#Define loss and optimizer
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits,labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op=optimizer.minimize(loss_op)
#Evaluate model
correct_pred=tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()


In [17]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1,num_steps+1):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        sess.run(train_op,feed_dict={X:batch_x,Y:batch_y,keep_prob:dropout})
        if step%display_step==0 or step==1:
            loss,acc=sess.run([loss_op,accuracy],feed_dict={X:batch_x,
                                                           Y:batch_y,
                                                           keep_prob:1.0})
            print("Step"+str(step)+",Minibatch Loss="+\
                 "{:.4f}".format(loss)+",Training Accuracy="+\
                 "{:.3f}".format(acc))
    print("Optimization Finished!")
    print("Testing Accuracy:",\
         sess.run(accuracy,feed_dict={X:mnist.test.images[:256],
                                     Y:mnist.test.labels[:256],
                                     keep_prob:1.0}))

Step1,Minibatch Loss=78430.3906,Training Accuracy=0.133
Step10,Minibatch Loss=27222.5938,Training Accuracy=0.172
Step20,Minibatch Loss=15743.8262,Training Accuracy=0.422
Step30,Minibatch Loss=8368.8506,Training Accuracy=0.578
Step40,Minibatch Loss=6488.8955,Training Accuracy=0.727
Step50,Minibatch Loss=5723.1040,Training Accuracy=0.719
Step60,Minibatch Loss=2823.7346,Training Accuracy=0.875
Step70,Minibatch Loss=3992.3042,Training Accuracy=0.781
Step80,Minibatch Loss=2144.1958,Training Accuracy=0.891
Step90,Minibatch Loss=1750.5883,Training Accuracy=0.883
Step100,Minibatch Loss=2059.6582,Training Accuracy=0.859
Step110,Minibatch Loss=2069.3232,Training Accuracy=0.844
Step120,Minibatch Loss=1371.9783,Training Accuracy=0.914
Step130,Minibatch Loss=1356.2539,Training Accuracy=0.930
Step140,Minibatch Loss=2636.4849,Training Accuracy=0.859
Step150,Minibatch Loss=1352.3926,Training Accuracy=0.891
Step160,Minibatch Loss=2236.0269,Training Accuracy=0.883
Step170,Minibatch Loss=2614.1062,Traini